<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/05_readmission_risk_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 본 프로젝트는 단계별 분석 파이프라인 형태로 구성되어 있으며 지금까지의 각 노트북은 다음과 같은 역할을 수행하였다.

## 01_data_loading.ipynb
- MIMIC-III Demo 데이터의 주요 테이블(PATIENTS, ADMISSIONS, ICUSTAYS 등)을 로드하고
병원 임상 데이터의 기본 구조와 테이블 간 관계를 확인하였다.

- 이후 코호트 정의와 분석을 위한 데이터 이해 단계에 해당한다.

---

## 02_cohort_definition.ipynb
- ICU에 입원한 환자 중 환자별 첫 입원(first admission)을 기준으로 분석 대상 cohort를 정의하였다.

- 입·퇴원 시점을 활용하여 30일 이내 재입원(readmission) 라벨을 생성하고
최종 분석용 데이터셋(**first_icu_admissions.csv**)을 구축하였다.

---

## 03_readmission_eda.ipynb
- 생성된 cohort를 대상으로 재입원 여부에 따른 기본적인 분포와 특성을 탐색하였다.

- 재입원 환자와 비재입원 환자 간의 차이를 인구학적 변수 및 입원 관련 변수 중심으로 확인하였다.

---

## 04_baseline_model.ipynb
- 제한된 기본 변수(age, ICU LOS)를 사용하여 30일 재입원 여부를 예측하는 간단한 baseline 모델을 구축하였다.

- 본 단계의 목적은 예측 성능 극대화가 아닌
**단순 변수 구성에서 얻을 수 있는 성능의 기준점(baseline)**을 설정하는 데 있다.

---

## 05_readmission_risk_factors.ipynb
- 위 단계에서 구축된 데이터와 분석 결과를 바탕으로
재입원 여부와 관련된 주요 위험 요인을 보다 집중적으로 비교·해석한다.

- 본 노트북에서는 앞선 단계에서 정의한 ICU 환자 cohort와
30일 이내 재입원(readmission) 라벨을 기반으로
재입원 여부와 관련된 주요 위험 요인(risk factors)을 분석한다.

- 단순히 예측 성능을 높이는 것을 목표로 하기보다는
재입원 환자와 비재입원 환자 간의 특성 차이를 비교하고
어떤 환자 및 입원 관련 변수들이 재입원 위험과 연관되어 있는지를
병원 임상 데이터 관점에서 해석하는 데 중점을 둔다.

In [1]:
# colab session setup
%cd /content
!rm -rf mimic-readmission-analysis
!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git
%cd mimic-readmission-analysis

/content
Cloning into 'mimic-readmission-analysis'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 215 (delta 37), reused 5 (delta 5), pack-reused 155 (from 1)
Receiving objects: 100% (215/215), 544.23 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/mimic-readmission-analysis


In [2]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")